In [ ]:
import polars as pl

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = "/content/drive/MyDrive/recommendation dataset"
user_pl  = pl.read_parquet(f"{data_path}/sales_pers.user_chunk_*.parquet")
item_pl  = pl.read_parquet(f"{data_path}/sales_pers.item_chunk_*.parquet")
purchase_pl = pl.read_parquet(f"{data_path}/sales_pers.purchase_history_daily_chunk_*.parquet")


In [ ]:
def print_pl_info(pl, name, show_all=False):
    print("="*80)
    print(f"\n✅ BẢNG: {name}")
    print("="*80)

    print(f"\n🔹 Shape: {pl.shape}")
    print("\n🔹Head")
    print(pl.head())

    print("\n🔹 Tổng quan thống kê:")
    display(pl.describe())

    print("\n🔹 Tên cột:")
    for name, dtype in pl.schema.items():
      print(f"{name:<20} {dtype}")

In [ ]:
def print_column_info(df: pl.DataFrame, col: str):
    series = df[col]

    print(f"\n🔹 Cột: {col}")
    print(f"Kiểu dữ liệu: {series.dtype}")
    print(f"Số dòng: {len(series)}")


    null_count = series.null_count()
    null_ratio = null_count / len(series) * 100

    print(f"Số lượng null: {null_count}")
    print(f"Tỷ lệ null: {null_ratio:.2f}%")


    unique_count = series.n_unique()
    print(f"Số lượng unique: {unique_count}")


    vc = series.value_counts(sort=True)


    count_col = "counts" if "counts" in vc.columns else "count"

    top10 = (
        vc.with_columns((pl.col(count_col) / len(series) * 100).alias("ratio (%)"))
        .head(10)
    )

    print("Top giá trị phổ biến:")
    print(top10)


In [ ]:
def plot_column(df, column):
    if df[column].dtype in [pl.Float64, pl.Int64, pl.Float32, pl.Int32]:
        plt.figure(figsize=(8, 4))
        sns.histplot(df[column].to_numpy(), bins=30, kde=True)
        plt.title(f"Distribution of {column}")
        plt.xlabel(column)
        plt.ylabel("Frequency")
        plt.show()


    else:

        counts = df[column].value_counts().sort("count", descending=True).head(10)


        counts_pd = counts.to_pandas()

        plt.figure(figsize=(8, 4))
        sns.barplot(
            data=counts_pd,
            x=column,
            y="count",
            palette="viridis",
        )
        plt.title(f"Top  values of {column}")
        plt.xticks(rotation=45)
        plt.xlabel(column)
        plt.ylabel("Count")
        plt.show()

In [ ]:
print_pl_info(user_pl,"USER")


✅ BẢNG: USER

🔹 Shape: (4573964, 18)

🔹Head
shape: (5, 18)
┌────────────┬────────┬──────────┬────────────┬───┬────────────┬──────────┬────────────┬───────────┐
│ customer_i ┆ gender ┆ location ┆ province   ┆ … ┆ install_da ┆ district ┆ user_id    ┆ is_delete │
│ d          ┆ ---    ┆ ---      ┆ ---        ┆   ┆ te         ┆ ---      ┆ ---        ┆ d         │
│ ---        ┆ str    ┆ i32      ┆ str        ┆   ┆ ---        ┆ str      ┆ str        ┆ ---       │
│ i32        ┆        ┆          ┆            ┆   ┆ i64        ┆          ┆            ┆ bool      │
╞════════════╪════════╪══════════╪════════════╪═══╪════════════╪══════════╪════════════╪═══════════╡
│ 14732      ┆ Nam    ┆ 155      ┆ Hồ Chí     ┆ … ┆ 1306281600 ┆ 7        ┆ e1e4820665 ┆ false     │
│            ┆        ┆          ┆ Minh       ┆   ┆            ┆          ┆ 2bf8c279ff ┆           │
│            ┆        ┆          ┆            ┆   ┆            ┆          ┆ 0206c69a80 ┆           │
│            ┆        ┆        

statistic,customer_id,gender,location,province,membership,timestamp,created_date,updated_date,sync_status_id,last_sync_date,sync_error_message,region,location_name,install_app,install_date,district,user_id,is_deleted
str,f64,str,f64,str,str,f64,str,str,f64,str,str,str,str,str,f64,str,str,f64
"""count""",4.573964e6,"""4573964""",4.573964e6,"""4573964""","""4573964""",4.573964e6,"""4573964""","""4573964""",4.235679e6,"""4235679""","""0""","""4573964""","""4573964""","""4573964""",4.573964e6,"""4573964""","""4573964""",4.573964e6
"""null_count""",0.0,"""0""",0.0,"""0""","""0""",0.0,"""0""","""0""",338285.0,"""338285""","""4573964""","""0""","""0""","""0""",0.0,"""0""","""0""",0.0
"""mean""",6.1266e6,null,554.053518,null,null,1.6712e9,"""2022-12-16 22:48:44.025646""","""2025-06-11 05:52:40.770554""",2.0,"""2025-07-16 15:20:34.188640""",null,null,null,null,1.6841e9,null,null,0.0
"""std""",2.4412e6,null,295.523361,null,null,7.9743e7,null,null,0.0,null,null,null,null,null,7.1374e7,null,null,null
"""min""",14732.0,"""Khác""",42.0,"""An Giang""","""Diamond""",1.3064e9,"""2011-05-25 21:11:51.677000""","""2019-06-30 08:38:47.100000""",2.0,"""2025-07-16 11:54:29.816986""",null,"""Bắc Trung Bộ""","""AGI - 110 Quốc Lộ 91""","""Android""",1.3063e9,""" Vũng Liêm""","""000004697bc1f0ecab42b6a3c2f0bc…",0.0
"""25%""",4.53268e6,null,302.0,null,null,1.6343e9,"""2021-10-15 10:09:34.280000""","""2025-07-07 15:33:10.201316""",2.0,"""2025-07-16 11:58:58.022193""",null,null,null,null,1.6550e9,null,null,null
"""50%""",6.843955e6,null,547.0,null,null,1.6960e9,"""2023-09-29 16:33:25.217000""","""2025-07-07 15:33:10.201316""",2.0,"""2025-07-16 12:03:34.550186""",null,null,null,null,1.7054e9,null,null,null
"""75%""",8.029649e6,null,765.0,null,null,1.7312e9,"""2024-11-09 14:38:22.223000""","""2025-08-02 10:54:03.860000""",2.0,"""2025-07-16 12:11:53.888687""",null,null,null,null,1.7366e9,null,null,null
"""max""",9.264159e6,"""Nữ""",1310.0,"""Đồng Tháp""","""Standard""",1.7593e9,"""2025-09-30 22:00:36.623000""","""2025-09-30 23:22:08.957000""",2.0,"""2025-07-21 11:19:29.058389""",null,"""Đồng bằng sông Cửu Long""","""ĐTH - Trần Hưng Đạo""","""iOS""",1.7592e9,"""Ứng Hòa""","""fffffbd469d500229dd000e9fc48ed…",0.0



🔹 Tên cột:
customer_id          Int32
gender               String
location             Int32
province             String
membership           String
timestamp            Int64
created_date         Datetime(time_unit='us', time_zone=None)
updated_date         Datetime(time_unit='us', time_zone=None)
sync_status_id       Int32
last_sync_date       Datetime(time_unit='us', time_zone=None)
sync_error_message   String
region               String
location_name        String
install_app          String
install_date         Int64
district             String
user_id              String
is_deleted           Boolean


In [ ]:
print_pl_info(item_pl,"ITEM")


✅ BẢNG: ITEM

🔹 Shape: (27332, 34)

🔹Head
shape: (5, 34)
┌───────┬────────────┬────────────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ p_id  ┆ item_id    ┆ price      ┆ category_l ┆ … ┆ volume    ┆ material  ┆ sale_stat ┆ descripti │
│ ---   ┆ ---        ┆ ---        ┆ 1_id       ┆   ┆ ---       ┆ ---       ┆ us        ┆ on_new    │
│ i32   ┆ str        ┆ decimal[38 ┆ ---        ┆   ┆ str       ┆ str       ┆ ---       ┆ ---       │
│       ┆            ┆ ,4]        ┆ i32        ┆   ┆           ┆           ┆ i32       ┆ str       │
╞═══════╪════════════╪════════════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 17065 ┆ 0502020000 ┆ 99000.0000 ┆ 1          ┆ … ┆ Không xác ┆ Không xác ┆ 0         ┆ Chi tiết  │
│       ┆ 004        ┆            ┆            ┆   ┆ định      ┆ định      ┆           ┆ sản phẩm  │
│       ┆            ┆            ┆            ┆   ┆           ┆           ┆           ┆ …         │
│ 72370 ┆ 0010290040 ┆ 69000.0000

statistic,p_id,item_id,price,category_l1_id,category_l1,category_l2_id,category_l2,category_l3_id,category_l3,category_id,category,description,brand,manufacturer,creation_timestamp,is_deleted,created_date,updated_date,sync_status_id,last_sync_date,sync_error_message,image_url,gender_target,age_group,item_type,gp,weight,color,size,origin,volume,material,sale_status,description_new
str,f64,str,f64,f64,str,f64,str,f64,str,f64,str,str,str,str,f64,f64,str,str,f64,str,str,str,str,str,str,f64,f64,str,str,str,str,str,f64,str
"""count""",27332.0,"""27332""",27332.0,27332.0,"""27332""",27332.0,"""27332""",27332.0,"""27332""",27332.0,"""27332""","""27332""","""27332""","""27332""",27332.0,27332.0,"""27332""","""27332""",26546.0,"""26546""","""0""","""27332""","""27332""","""27332""","""27332""",27332.0,0.0,"""27332""","""27332""","""27332""","""27332""","""27332""",27332.0,"""22317"""
"""null_count""",0.0,"""0""",0.0,0.0,"""0""",0.0,"""0""",0.0,"""0""",0.0,"""0""","""0""","""0""","""0""",0.0,0.0,"""0""","""0""",786.0,"""786""","""27332""","""0""","""0""","""0""","""0""",0.0,27332.0,"""0""","""0""","""0""","""0""","""0""",0.0,"""5015"""
"""mean""",106317.679643,null,190456.829467,2836.189192,null,3719.039843,null,4876.768294,null,4920.712718,null,null,null,null,1.6225e9,0.0,"""2021-05-31 17:17:33.849833""","""2025-08-30 04:07:07.766157""",2.0,"""2025-07-18 18:00:25.086391""",null,null,null,null,null,49696.651226,null,null,null,null,null,null,0.167203,null
"""std""",25425.734073,null,511123.971813,1417.521137,null,1935.042044,null,2035.952741,null,1999.308355,null,null,null,null,7.5666e7,null,null,null,0.0,null,null,null,null,null,null,144155.68753,null,null,null,null,null,null,0.373164,null
"""min""",1675.0,"""0000280000019""",0.0,1.0,"""Babycare""",2.0,"""0-1Y""",14.0,"""A2 milk""",4.0,"""0-12M Bodysuit bé trai đùi""","""Với kiểu dáng đơn giản nhưng …","""137 Degrees""","""(L) Địa điểm kinh doanh 1 - Ch…",1.2983e9,0.0,"""2011-02-21 11:54:02.047000""","""2023-12-26 17:21:11.663000""",2.0,"""2025-07-18 17:59:29.898256""",null,"""Không xác định""","""Bé Gái""","""0-10M""","""BCS""",0.0,null,"""Bạc""","""0 tháng""",""" Singapore, Thụy Sĩ""","""1 lít""",""" Vải không dệt, bông cellulose…",0.0,"""Chi tiết sản phẩm …"
"""25%""",88905.0,null,49000.0,2788.0,null,2038.0,null,3516.0,null,3522.0,null,null,null,null,1.5755e9,null,"""2019-12-04 18:24:16.260000""","""2025-08-18 09:59:19.847000""",2.0,"""2025-07-18 17:59:29.898256""",null,null,null,null,null,0.0,null,null,null,null,null,null,0.0,null
"""50%""",104510.0,null,119000.0,3292.0,null,3958.0,null,5254.0,null,5511.0,null,null,null,null,1.6358e9,null,"""2021-11-01 13:19:26.080000""","""2025-09-26 08:05:23.153000""",2.0,"""2025-07-18 17:59:29.898256""",null,null,null,null,null,25047.0,null,null,null,null,null,null,0.0,null
"""75%""",120262.0,null,199000.0,3292.0,null,4571.0,null,6984.0,null,6988.0,null,null,null,null,1.6699e9,null,"""2022-12-01 15:40:42.260000""","""2025-09-27 00:05:36.233000""",2.0,"""2025-07-18 17:59:29.898256""",null,null,null,null,null,65532.0,null,null,null,null,null,null,0.0,null
"""max""",167147.0,"""7497000000006""",2.099e7,7376.0,"""Đồ chơi & Sách""",7492.0,"""Đồ uống""",7493.0,"""Đồ hộp""",7501.0,"""Động vật mô hình""","""﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿﻿Bỉm …","""Đức Thành""","""﻿Shantou City Chenghai Jia Ron…",1.7559e9,0.0,"""2025-08-22 17:35:14.377000""","""2025-10-01 08:05:42.990000""",2.0,"""2025-07-21 10:59:04.045058""",null,"""Không xác định""","""Unisex""","""[""Từ 6M"", ""Từ 9M"", ""Từ 1Y""]""","""Ứng dụng mẹ và bé""",5.54136e6,null,"""Đỏ""","""XXL(15-25kg) - 26 miếng""","""Ấn Độ, Bulgaria""","""hộp 3 lọ x 250ml""","""﻿100% cotton﻿<br>""",1.0,"""Xin lỗi, nhưng tôi không thể t…"



🔹 Tên cột:
p_id                 Int32
item_id              String
price                Decimal(precision=38, scale=4)
category_l1_id       Int32
category_l1          String
category_l2_id       Int32
category_l2          String
category_l3_id       Int32
category_l3          String
category_id          Int32
category             String
description          String
brand                String
manufacturer         String
creation_timestamp   Int64
is_deleted           Boolean
created_date         Datetime(time_unit='us', time_zone=None)
updated_date         Datetime(time_unit='us', time_zone=None)
sync_status_id       Int32
last_sync_date       Datetime(time_unit='us', time_zone=None)
sync_error_message   String
image_url            String
gender_target        String
age_group            String
item_type            String
gp                   Decimal(precision=38, scale=4)
weight               Float32
color                String
size                 String
origin               String
vol

In [ ]:
print_pl_info(purchase_pl,"PURCHASE")


✅ BẢNG: PURCHASE

🔹 Shape: (35729825, 16)

🔹Head
shape: (5, 16)
┌────────────┬───────────┬───────────┬───────────┬───┬──────────┬──────────┬───────────┬───────────┐
│ timestamp  ┆ user_id   ┆ item_id   ┆ event_typ ┆ … ┆ payment  ┆ location ┆ discount  ┆ is_delete │
│ ---        ┆ ---       ┆ ---       ┆ e         ┆   ┆ ---      ┆ ---      ┆ ---       ┆ d         │
│ i64        ┆ str       ┆ str       ┆ ---       ┆   ┆ str      ┆ i32      ┆ decimal[3 ┆ ---       │
│            ┆           ┆           ┆ str       ┆   ┆          ┆          ┆ 8,4]      ┆ bool      │
╞════════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪══════════╪═══════════╪═══════════╡
│ 1735064221 ┆ ca12702dd ┆ 711500000 ┆ Purchase  ┆ … ┆ VietQR   ┆ 656      ┆ 0.0000    ┆ false     │
│            ┆ f55acaa9f ┆ 0004      ┆           ┆   ┆          ┆          ┆           ┆           │
│            ┆ b767e10fa ┆           ┆           ┆   ┆          ┆          ┆           ┆           │
│            ┆ aa6…      ┆

statistic,timestamp,user_id,item_id,event_type,event_value,price,date_key,quantity,customer_id,created_date,updated_date,channel,payment,location,discount,is_deleted
str,f64,str,str,str,f64,f64,f64,f64,f64,str,str,str,str,f64,f64,f64
"""count""",3.5729825e7,"""35729825""","""35729825""","""35729825""",3.5729825e7,3.5729825e7,3.5729825e7,3.5729825e7,3.5729825e7,"""35729825""","""35729825""","""35729825""","""35729825""",3.5729825e7,3.5729825e7,3.5729825e7
"""null_count""",0.0,"""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0
"""mean""",1.7199e9,null,null,null,1.508098,169188.742494,2.0241e7,1.508098,4.8713e6,"""2024-07-02 17:13:24.220955""","""2024-07-02 18:27:59.561504""",null,null,448.414508,17235.651914,0.0
"""std""",9.1568e6,null,null,null,2.264976,195539.88976,346.231465,2.264976,2.2697e6,null,null,null,null,247.505432,46673.708499,null
"""min""",1.7041e9,"""0000063a29e92d643437aa09266fdc…","""0000280000019""","""Purchase""",1.0,0.0444,2.0240101e7,1.0,14732.0,"""2024-01-01 06:44:59.037000""","""2024-01-01 06:44:59.233000""","""Android""","""Chuyển khoản""",16.0,0.0,0.0
"""25%""",1.7120e9,null,null,null,1.0,48316.3914,2.0240401e7,1.0,3.06641e6,"""2024-04-01 16:44:10.513000""","""2024-04-01 17:15:12.017000""",null,null,234.0,0.0,null
"""50%""",1.7199e9,null,null,null,1.0,87000.0,2.0240702e7,1.0,5.373736e6,"""2024-07-02 19:06:52.293000""","""2024-07-02 19:42:24.550000""",null,null,439.0,0.0,null
"""75%""",1.7279e9,null,null,null,1.0,244000.0,2.0241002e7,1.0,6.853734e6,"""2024-10-02 19:55:22.447000""","""2024-10-02 20:24:53.813000""",null,null,653.0,15000.0,null
"""max""",1.7357e9,"""fffff7e197ebf5c523fa686e6a305c…","""7352000000004""","""Purchase""",240.0,2.099e7,2.0241231e7,240.0,8.208027e6,"""2024-12-31 22:35:19.510000""","""2025-09-27 21:25:22.960000""","""iOS""","""ZaloPay""",1036.0,7.995e6,0.0



🔹 Tên cột:
timestamp            Int64
user_id              String
item_id              String
event_type           String
event_value          Decimal(precision=38, scale=4)
price                Decimal(precision=38, scale=4)
date_key             Int32
quantity             Int32
customer_id          Int32
created_date         Datetime(time_unit='us', time_zone=None)
updated_date         Datetime(time_unit='us', time_zone=None)
channel              String
payment              String
location             Int32
discount             Decimal(precision=38, scale=4)
is_deleted           Boolean


#TIỀN XỬ LÝ DỮ LIỆU

In [ ]:
user_pl_pre = user_pl.drop(["sync_error_message", "sync_status_id", "is_deleted" , "timestamp","location_name"])

In [ ]:

string_cols = ["gender", "province", "membership", "install_app"]

user_pl_pre = user_pl_pre.with_columns([
    pl.col(col).cast(pl.Categorical).to_physical().alias(f"{col}_encoded")
    for col in string_cols
])


In [ ]:
print_pl_info(user_pl_pre , "USER")


✅ BẢNG: USER

🔹 Shape: (4573964, 17)

🔹Head
shape: (5, 17)
┌────────────┬────────┬──────────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
│ customer_i ┆ gender ┆ location ┆ province   ┆ … ┆ gender_enc ┆ province_ ┆ membershi ┆ install_a │
│ d          ┆ ---    ┆ ---      ┆ ---        ┆   ┆ oded       ┆ encoded   ┆ p_encoded ┆ pp_encode │
│ ---        ┆ str    ┆ i32      ┆ str        ┆   ┆ ---        ┆ ---       ┆ ---       ┆ d         │
│ i32        ┆        ┆          ┆            ┆   ┆ u32        ┆ u32       ┆ u32       ┆ ---       │
│            ┆        ┆          ┆            ┆   ┆            ┆           ┆           ┆ u32       │
╞════════════╪════════╪══════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
│ 14732      ┆ Nam    ┆ 155      ┆ Hồ Chí     ┆ … ┆ 0          ┆ 0         ┆ 0         ┆ 0         │
│            ┆        ┆          ┆ Minh       ┆   ┆            ┆           ┆           ┆           │
│ 15126      ┆ Nữ     ┆ 300    

statistic,customer_id,gender,location,province,membership,created_date,updated_date,last_sync_date,region,install_app,install_date,district,user_id,gender_encoded,province_encoded,membership_encoded,install_app_encoded
str,f64,str,f64,str,str,str,str,str,str,str,f64,str,str,f64,f64,f64,f64
"""count""",4.573964e6,"""4573964""",4.573964e6,"""4573964""","""4573964""","""4573964""","""4573964""","""4235679""","""4573964""","""4573964""",4.573964e6,"""4573964""","""4573964""",4.573964e6,4.573964e6,4.573964e6,4.573964e6
"""null_count""",0.0,"""0""",0.0,"""0""","""0""","""0""","""0""","""338285""","""0""","""0""",0.0,"""0""","""0""",0.0,0.0,0.0,0.0
"""mean""",6.1266e6,null,554.053518,null,null,"""2022-12-16 22:48:44.025646""","""2025-06-11 05:52:40.770554""","""2025-07-16 15:20:34.188640""",null,null,1.6841e9,null,null,0.748782,13.662971,0.088952,0.325703
"""std""",2.4412e6,null,295.523361,null,null,null,null,null,null,null,7.1374e7,null,null,0.433718,14.712908,0.33794,1.058428
"""min""",14732.0,"""Khác""",42.0,"""An Giang""","""Diamond""","""2011-05-25 21:11:51.677000""","""2019-06-30 08:38:47.100000""","""2025-07-16 11:54:29.816986""","""Bắc Trung Bộ""","""Android""",1.3063e9,""" Vũng Liêm""","""000004697bc1f0ecab42b6a3c2f0bc…",0.0,0.0,0.0,0.0
"""25%""",4.53268e6,null,302.0,null,null,"""2021-10-15 10:09:34.280000""","""2025-07-07 15:33:10.201316""","""2025-07-16 11:58:58.022193""",null,null,1.6550e9,null,null,0.0,0.0,0.0,0.0
"""50%""",6.843955e6,null,547.0,null,null,"""2023-09-29 16:33:25.217000""","""2025-07-07 15:33:10.201316""","""2025-07-16 12:03:34.550186""",null,null,1.7054e9,null,null,1.0,8.0,0.0,0.0
"""75%""",8.029649e6,null,765.0,null,null,"""2024-11-09 14:38:22.223000""","""2025-08-02 10:54:03.860000""","""2025-07-16 12:11:53.888687""",null,null,1.7366e9,null,null,1.0,23.0,0.0,0.0
"""max""",9.264159e6,"""Nữ""",1310.0,"""Đồng Tháp""","""Standard""","""2025-09-30 22:00:36.623000""","""2025-09-30 23:22:08.957000""","""2025-07-21 11:19:29.058389""","""Đồng bằng sông Cửu Long""","""iOS""",1.7592e9,"""Ứng Hòa""","""fffffbd469d500229dd000e9fc48ed…",2.0,125.0,2.0,10.0



🔹 Tên cột:
customer_id          Int32
gender               String
location             Int32
province             String
membership           String
created_date         Datetime(time_unit='us', time_zone=None)
updated_date         Datetime(time_unit='us', time_zone=None)
last_sync_date       Datetime(time_unit='us', time_zone=None)
region               String
install_app          String
install_date         Int64
district             String
user_id              String
gender_encoded       UInt32
province_encoded     UInt32
membership_encoded   UInt32
install_app_encoded  UInt32


In [ ]:

item_pl_pre = item_pl.drop([
    "category" ,
    "category_l1" ,
    "category_l2" ,
    "category_l3",
    "description",
    "manufacturer",
    "is_deleted",
    "sync_status_id",
    "last_sync_date",
    "sync_error_message",
    "image_url",
    "weight",
    "color",
    "size",
    "origin",
    "sale_status",
    "description_new",
    "volume",
    "material"
])


In [ ]:

string_cols = ["brand", "gender_target", "age_group", "item_type"]


item_pl_pre = item_pl_pre.with_columns([
    pl.col(col).cast(pl.Categorical).to_physical().alias(f"{col}_encoded")
    for col in string_cols
])


In [ ]:
print_pl_info(item_pl_pre , "ITEM")


✅ BẢNG: ITEM

🔹 Shape: (27332, 19)

🔹Head
shape: (5, 19)
┌───────┬────────────┬────────────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ p_id  ┆ item_id    ┆ price      ┆ category_l ┆ … ┆ brand_enc ┆ gender_ta ┆ age_group ┆ item_type │
│ ---   ┆ ---        ┆ ---        ┆ 1_id       ┆   ┆ oded      ┆ rget_enco ┆ _encoded  ┆ _encoded  │
│ i32   ┆ str        ┆ decimal[38 ┆ ---        ┆   ┆ ---       ┆ ded       ┆ ---       ┆ ---       │
│       ┆            ┆ ,4]        ┆ i32        ┆   ┆ u32       ┆ ---       ┆ u32       ┆ u32       │
│       ┆            ┆            ┆            ┆   ┆           ┆ u32       ┆           ┆           │
╞═══════╪════════════╪════════════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 17065 ┆ 0502020000 ┆ 99000.0000 ┆ 1          ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ 0         │
│       ┆ 004        ┆            ┆            ┆   ┆           ┆           ┆           ┆           │
│ 72370 ┆ 0010290040 ┆ 69000.0000

statistic,p_id,item_id,price,category_l1_id,category_l2_id,category_l3_id,category_id,brand,creation_timestamp,created_date,updated_date,gender_target,age_group,item_type,gp,brand_encoded,gender_target_encoded,age_group_encoded,item_type_encoded
str,f64,str,f64,f64,f64,f64,f64,str,f64,str,str,str,str,str,f64,f64,f64,f64,f64
"""count""",27332.0,"""27332""",27332.0,27332.0,27332.0,27332.0,27332.0,"""27332""",27332.0,"""27332""","""27332""","""27332""","""27332""","""27332""",27332.0,27332.0,27332.0,27332.0,27332.0
"""null_count""",0.0,"""0""",0.0,0.0,0.0,0.0,0.0,"""0""",0.0,"""0""","""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0,0.0
"""mean""",106317.679643,null,190456.829467,2836.189192,3719.039843,4876.768294,4920.712718,null,1.6225e9,"""2021-05-31 17:17:33.849833""","""2025-08-30 04:07:07.766157""",null,null,null,49696.651226,79.996341,0.598346,8.33898,20.548771
"""std""",25425.734073,null,511123.971813,1417.521137,1935.042044,2035.952741,1999.308355,null,7.5666e7,null,null,null,null,null,144155.68753,173.132202,0.945801,13.837149,34.693609
"""min""",1675.0,"""0000280000019""",0.0,1.0,2.0,14.0,4.0,"""137 Degrees""",1.2983e9,"""2011-02-21 11:54:02.047000""","""2023-12-26 17:21:11.663000""","""Bé Gái""","""0-10M""","""BCS""",0.0,0.0,0.0,0.0,0.0
"""25%""",88905.0,null,49000.0,2788.0,2038.0,3516.0,3522.0,null,1.5755e9,"""2019-12-04 18:24:16.260000""","""2025-08-18 09:59:19.847000""",null,null,null,0.0,7.0,0.0,0.0,0.0
"""50%""",104510.0,null,119000.0,3292.0,3958.0,5254.0,5511.0,null,1.6358e9,"""2021-11-01 13:19:26.080000""","""2025-09-26 08:05:23.153000""",null,null,null,25047.0,19.0,0.0,0.0,1.0
"""75%""",120262.0,null,199000.0,3292.0,4571.0,6984.0,6988.0,null,1.6699e9,"""2022-12-01 15:40:42.260000""","""2025-09-27 00:05:36.233000""",null,null,null,65532.0,19.0,1.0,14.0,30.0
"""max""",167147.0,"""7497000000006""",2.099e7,7376.0,7492.0,7493.0,7501.0,"""Đức Thành""",1.7559e9,"""2025-08-22 17:35:14.377000""","""2025-10-01 08:05:42.990000""","""Unisex""","""[""Từ 6M"", ""Từ 9M"", ""Từ 1Y""]""","""Ứng dụng mẹ và bé""",5.54136e6,975.0,4.0,111.0,194.0



🔹 Tên cột:
p_id                 Int32
item_id              String
price                Decimal(precision=38, scale=4)
category_l1_id       Int32
category_l2_id       Int32
category_l3_id       Int32
category_id          Int32
brand                String
creation_timestamp   Int64
created_date         Datetime(time_unit='us', time_zone=None)
updated_date         Datetime(time_unit='us', time_zone=None)
gender_target        String
age_group            String
item_type            String
gp                   Decimal(precision=38, scale=4)
brand_encoded        UInt32
gender_target_encoded UInt32
age_group_encoded    UInt32
item_type_encoded    UInt32


In [ ]:
purchase_pl_pre = purchase_pl.drop([
    "event_type",
    "event_value",
    "is_deleted"
])


In [ ]:
purchase_pl_pre = purchase_pl_pre.with_columns([
    pl.col("channel").cast(pl.Categorical).to_physical().alias("channel_encoded"),
    pl.col("payment").cast(pl.Categorical).to_physical().alias("payment_encoded")
])


In [ ]:
print_pl_info(purchase_pl_pre, "PURCHASE")


✅ BẢNG: PURCHASE

🔹 Shape: (35729825, 15)

🔹Head
shape: (5, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬──────────┬───────────┬───────────┬───────────┐
│ timestamp ┆ user_id   ┆ item_id   ┆ price     ┆ … ┆ location ┆ discount  ┆ channel_e ┆ payment_e │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---      ┆ ---       ┆ ncoded    ┆ ncoded    │
│ i64       ┆ str       ┆ str       ┆ decimal[3 ┆   ┆ i32      ┆ decimal[3 ┆ ---       ┆ ---       │
│           ┆           ┆           ┆ 8,4]      ┆   ┆          ┆ 8,4]      ┆ u32       ┆ u32       │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪══════════╪═══════════╪═══════════╪═══════════╡
│ 173506422 ┆ ca12702dd ┆ 711500000 ┆ 49000.000 ┆ … ┆ 656      ┆ 0.0000    ┆ 0         ┆ 0         │
│ 1         ┆ f55acaa9f ┆ 0004      ┆ 0         ┆   ┆          ┆           ┆           ┆           │
│           ┆ b767e10fa ┆           ┆           ┆   ┆          ┆           ┆           ┆           │
│           ┆ aa6…      ┆ 

statistic,timestamp,user_id,item_id,price,date_key,quantity,customer_id,created_date,updated_date,channel,payment,location,discount,channel_encoded,payment_encoded
str,f64,str,str,f64,f64,f64,f64,str,str,str,str,f64,f64,f64,f64
"""count""",3.5729825e7,"""35729825""","""35729825""",3.5729825e7,3.5729825e7,3.5729825e7,3.5729825e7,"""35729825""","""35729825""","""35729825""","""35729825""",3.5729825e7,3.5729825e7,3.5729825e7,3.5729825e7
"""null_count""",0.0,"""0""","""0""",0.0,0.0,0.0,0.0,"""0""","""0""","""0""","""0""",0.0,0.0,0.0,0.0
"""mean""",1.7199e9,null,null,169188.742494,2.0241e7,1.508098,4.8713e6,"""2024-07-02 17:13:24.220955""","""2024-07-02 18:27:59.561504""",null,null,448.414508,17235.651914,0.183794,1.417781
"""std""",9.1568e6,null,null,195539.88976,346.231465,2.264976,2.2697e6,null,null,null,null,247.505432,46673.708499,0.712921,1.371772
"""min""",1.7041e9,"""0000063a29e92d643437aa09266fdc…","""0000280000019""",0.0444,2.0240101e7,1.0,14732.0,"""2024-01-01 06:44:59.037000""","""2024-01-01 06:44:59.233000""","""Android""","""Chuyển khoản""",16.0,0.0,0.0,0.0
"""25%""",1.7120e9,null,null,48316.3914,2.0240401e7,1.0,3.06641e6,"""2024-04-01 16:44:10.513000""","""2024-04-01 17:15:12.017000""",null,null,234.0,0.0,0.0,1.0
"""50%""",1.7199e9,null,null,87000.0,2.0240702e7,1.0,5.373736e6,"""2024-07-02 19:06:52.293000""","""2024-07-02 19:42:24.550000""",null,null,439.0,0.0,0.0,1.0
"""75%""",1.7279e9,null,null,244000.0,2.0241002e7,1.0,6.853734e6,"""2024-10-02 19:55:22.447000""","""2024-10-02 20:24:53.813000""",null,null,653.0,15000.0,0.0,1.0
"""max""",1.7357e9,"""fffff7e197ebf5c523fa686e6a305c…","""7352000000004""",2.099e7,2.0241231e7,240.0,8.208027e6,"""2024-12-31 22:35:19.510000""","""2025-09-27 21:25:22.960000""","""iOS""","""ZaloPay""",1036.0,7.995e6,10.0,9.0



🔹 Tên cột:
timestamp            Int64
user_id              String
item_id              String
price                Decimal(precision=38, scale=4)
date_key             Int32
quantity             Int32
customer_id          Int32
created_date         Datetime(time_unit='us', time_zone=None)
updated_date         Datetime(time_unit='us', time_zone=None)
channel              String
payment              String
location             Int32
discount             Decimal(precision=38, scale=4)
channel_encoded      UInt32
payment_encoded      UInt32


In [ ]:
purchase_pl_pre = purchase_pl_pre.lazy()

print("\nĐang tính tổng quan toàn bảng...")
total_stats = (
    purchase_pl_pre.select([
        pl.len().alias("total_transactions"),
        (pl.col("quantity") * pl.col("price")).sum().alias("total_revenue"),
        pl.col("item_id").n_unique().alias("total_unique_items"),
        pl.col("customer_id").n_unique().alias("total_unique_customers")
    ])
    .collect()
)

total_trans      = total_stats["total_transactions"][0]
total_rev        = total_stats["total_revenue"][0]
total_items      = total_stats["total_unique_items"][0]
total_customers  = total_stats["total_unique_customers"][0]

print(f"Toàn bảng gốc:")
print(f"   → {total_customers:,} khách hàng")
print(f"   → {total_trans:,} giao dịch")
print(f"   → {total_items:,} item_id khác nhau")
print(f"   → Doanh thu tổng: {total_rev:,.0f}")

print("\nĐang nhóm theo item_id và tính freq + revenue...")
print('\n')
item_stats = (
    purchase_pl_pre.group_by("item_id")
        .agg([
            pl.len().alias("freq"),   # tổng giao dịch
            (pl.col("quantity") * pl.col("price")).sum().alias("revenue"),
            pl.col("customer_id").n_unique().alias("num_customers")  # ✅ số khách hàng duy nhất
        ])
        .sort("freq", descending=True)
        .collect()
)

print(item_stats.head())
top_items = [500, 1000, 2000, 2500, 3000]

for k in top_items:
    top = item_stats.head(k)
    top_item_ids = top["item_id"].to_list()  # Danh sách item_id trong top-k

    # Tính tổng freq và revenue
    top_summary = top.select([
        pl.sum("freq").alias("top_freq"),
        pl.sum("revenue").alias("top_revenue")
    ])

    top_freq = top_summary["top_freq"][0]
    top_rev  = top_summary["top_revenue"][0]


    pct_freq = (top_freq / total_trans) * 100
    pct_rev  = (top_rev / total_rev) * 100

    top_item_ids = top["item_id"].to_list()
    customer_covered = (
        purchase_pl_pre.filter(pl.col("item_id").is_in(top_item_ids))
        .select("customer_id")
        .unique()
        .collect()
        .height
    )
    pct_customer = customer_covered / total_customers * 100

    # In kết quả
    print(f"\nTOP {k} ITEM (chiếm {k}/{total_items} items = {k/total_items*100:.3f}%)")
    print(f"   → Số lần xuất hiện: {top_freq:,} → {pct_freq:.3f}% tổng giao dịch")
    print(f"   → Doanh thu       : {top_rev:,.0f} → {pct_rev:.3f}% tổng doanh thu")
    print(f"   → Độ phủ khách hàng: {customer_covered:,} ({pct_customer:.3f}%)\n")



Đang tính tổng quan toàn bảng...
Toàn bảng gốc:
   → 2,442,306 khách hàng
   → 35,729,825 giao dịch
   → 20,810 item_id khác nhau
   → Doanh thu tổng: 7,944,845,873,918

Đang nhóm theo item_id và tính freq + revenue...


shape: (5, 4)
┌───────────────┬────────┬───────────────────────┬───────────────┐
│ item_id       ┆ freq   ┆ revenue               ┆ num_customers │
│ ---           ┆ ---    ┆ ---                   ┆ ---           │
│ str           ┆ u32    ┆ decimal[*,8]          ┆ u32           │
╞═══════════════╪════════╪═══════════════════════╪═══════════════╡
│ 1512000000004 ┆ 302740 ┆ 98923793325.43230000  ┆ 189921        │
│ 2803000000013 ┆ 224091 ┆ 12009766472.48780000  ┆ 114458        │
│ 0020010000438 ┆ 206507 ┆ 145226044821.69450000 ┆ 57157         │
│ 4690000000001 ┆ 197920 ┆ 16216413426.96990000  ┆ 127451        │
│ 2803000000011 ┆ 185810 ┆ 9796883122.99410000   ┆ 94453         │
└───────────────┴────────┴───────────────────────┴───────────────┘

TOP 500 ITEM (chiếm 500/20

In [ ]:
# 1. Lấy danh sách 2500 item hot nhất
top_2500_items = item_stats.head(2500)["item_id"].to_list()
top_customer = customer_stats.head(1000000)["customer_id"].to_list()

# 2. LỌC SẠCH 2 BẢNG CHỈ CÒN 2500 ITEM NÀY
item_pl_pre = item_pl_pre.filter(pl.col("item_id").is_in(top_2500_items))

user_pl_pre = user_pl_pre.filter(pl.col("customer_id").is_in(top_customer))

purchase_pl_pre = (purchase_pl_pre
                   .filter(pl.col("item_id").is_in(top_2500_items))
                   .filter(pl.col("customer_id").is_in(top_customer)))



print(f"user_pl_pre: {user_pl_pre.shape}")
print(f"item_pl_pre: {item_pl_pre.shape}")
print(f"purchase_pl_pre: {purchase_pl_pre.collect().height} ")

NameError: name 'customer_stats' is not defined

In [ ]:
purchase_pl_pre = (
    purchase_pl_pre
    .sort("created_date")
)

Dựa vào các bảng dữ liệu đã cung cấp, tạo các đặc trưng sau:

- Đặc trưng sản phẩm mua cùng: | item A | itemB | #cooc |

- Đặc trưng lứa tuổi của em bé: | customer_id | date_of_birth | age_in_month |

- Đặc trưng phân khúc sản phẩm sữa Step 1:

| item_id | category | segment |

- Đặc trưng phân khúc khách hàng dựa trên sữa Step 1

| customer_id |  segment |

Các bạn thảo luận nhóm để brainstorm suy nghĩ xem đặc trưng nào có tính chất tuyến tính:

- Giá trị đặc trưng càng TĂNG thì khả năng customer_id mua item_id đó càng CAO

- Giá trị đặc trưng càng TĂNG thì khả năng customer_id mua item_id đó càng THẤP

Đây là bước quan trọng của đồ án, tiền đề để mô hình máy học có thể huấn luyện.
Mỗi nhóm tiến hành cài đặt hàm tạo bảng đặc trưng:
- Input:
    + LazyFrame: Transactions, Items, Users
    + Datetime: begin_hist
    + Datetime: end_hist
    + Datetime: begin_recent
    + Datetime: end_recent
- Output: Bảng Feature - Label gồm
    + X_-1: customer_id
    + X_0: item_id
    + X_1: số lần xuất hiện brand của item_id trong các item đã mua trong giai đoạn (begin_hist - end_hist)
    + X_2: số lần xuất hiện age_group của item_id trong các item đã mua trong giai đoạn (begin_hist - end_hist)
    + X_3: số lần xuất hiện category của item_id trong các item đã mua trong giai đoạn (begin_hist - end_hist)
    + Y: bằng 1 nếu customer_id và item_id CÓ GIAO DỊCH trong giai đoạn (begin_recent - end_recent), ngược lại bằng 0

Sử dụng bảng đặc trưng và groundtruth tuần trước để huấn luyện mô hình.
Sau đó, tính lại bảng đặc trưng test trong khoảng thời gian train_history và train_recent.
Áp dụng mô hình đã huấn luyện lên bảng đặc trưng test
Sắp xếp giảm dần giá trị dự đoán
Sau đó so sánh với groundtruth được lấy từ giai đoạn validation để tính precision@K và NDCG@K

Lỗi bị chỉ tính những vật phẩm đã mua trong thời gian và bảng train và test luôn là y = 1 lỗi quá dưới đây là code sửa vấn đề trên và khi mà sử dụng phần

In [ ]:
print_column_info(purchase_pl,"quantity")


🔹 Cột: quantity
Kiểu dữ liệu: Int32
Số dòng: 35729825
Số lượng null: 0
Tỷ lệ null: 0.00%
Số lượng unique: 90
Top giá trị phổ biến:
shape: (10, 3)
┌──────────┬──────────┬───────────┐
│ quantity ┆ count    ┆ ratio (%) │
│ ---      ┆ ---      ┆ ---       │
│ i32      ┆ u32      ┆ f64       │
╞══════════╪══════════╪═══════════╡
│ 1        ┆ 29446356 ┆ 82.413938 │
│ 2        ┆ 4044053  ┆ 11.318424 │
│ 3        ┆ 687091   ┆ 1.923018  │
│ 4        ┆ 473658   ┆ 1.325666  │
│ 12       ┆ 468496   ┆ 1.311218  │
│ 5        ┆ 215773   ┆ 0.603902  │
│ 6        ┆ 137964   ┆ 0.386131  │
│ 24       ┆ 72318    ┆ 0.202402  │
│ 8        ┆ 44466    ┆ 0.124451  │
│ 36       ┆ 27915    ┆ 0.078128  │
└──────────┴──────────┴───────────┘


In [ ]:
import polars as pl
from datetime import datetime
def create_feature_table(
    transactions: pl.LazyFrame,
    items: pl.LazyFrame,
    begin_hist: datetime,
    end_hist: datetime,
    begin_recent: datetime,
    end_recent: datetime,
) -> pl.DataFrame:

    hist = transactions.filter(
        (pl.col("created_date") >= begin_hist) &
        (pl.col("created_date") <= end_hist)
    )

    recent = transactions.filter(
        (pl.col("created_date") >= begin_recent) &
        (pl.col("created_date") <= end_recent)
    )

    pos = (
        recent
        .select(["customer_id", "item_id"])
        .unique()
        .with_columns(pl.lit(1).alias("Y"))
    )

    items_feat = items.select(
        ["item_id", "brand", "category_id"]
    )

    hist_join = hist.join(items_feat, on="item_id", how="left")

    brand_hist = (
        hist_join
        .group_by(["customer_id", "brand"])
        .agg(pl.len().alias("brand_freq"))
    )

    cate_hist = (
        hist_join
        .group_by(["customer_id", "category_id"])
        .agg(pl.len().alias("cate_freq"))
    )

    print(hist.limit(10).collect())
    print(brand_hist.limit(10).collect())
    print(cate_hist.limit(10).collect())

    pos_with_item = pos.join(items_feat, on="item_id", how="left")

    feature_table = (
        pos_with_item
        .join(brand_hist, on=["customer_id", "brand"], how="left")
        .join(cate_hist, on=["customer_id", "category_id"], how="left")
        .with_columns([
            pl.col("X1").fill_null(0),
            pl.col("X3").fill_null(0),
        ])
        .select([
            "customer_id",
            "item_id",
            "brand_freq",
            "cate_freq",
            "Y"
        ])
    )

    return feature_table.collect()


In [ ]:
print(df_train.head())
print(df_train.shape)
print(df_train.head())
print(df_train.shape)
print(df_train.describe())



NameError: name 'df_train' is not defined

In [ ]:
x = ["X1","X3"]

In [ ]:
print_pl_info(df_train,"train")

NameError: name 'df_train' is not defined

In [ ]:
for i in x:
    print_column_info(df_train,i)

In [ ]:
import polars as pl
from datetime import datetime

def create_feature_table(
    transactions: pl.LazyFrame,
    items: pl.LazyFrame,
    begin_hist: datetime,
    end_hist: datetime,
    begin_recent: datetime,
    end_recent: datetime,
) -> pl.DataFrame:

    # Lọc dữ liệu lịch sử và recent
    hist = transactions.filter(
        (pl.col("created_date") >= begin_hist) &
        (pl.col("created_date") <= end_hist)
    )

    recent = transactions.filter(
        (pl.col("created_date") >= begin_recent) &
        (pl.col("created_date") <= end_recent)
    )

    # Positive samples (Y=1)
    pos = (
        recent
        .select(["customer_id", "item_id"])
        .unique()
        .with_columns(pl.lit(1).alias("Y"))
    )



    return pos.collect()

In [ ]:
pos_train = create_feature_table(
    transactions = purchase_pl_pre.lazy(),
    items = item_pl_pre.lazy(),
    begin_hist     = datetime(2024, 1, 1),
    end_hist       = datetime(2024, 11, 30),
    begin_recent   = datetime(2024, 12, 1),
    end_recent     = datetime(2024, 12, 31)
)


In [ ]:
print_pl_info(pos_train,"train")

In [ ]:
import numpy as np


In [ ]:
pos_sampled = (
    pos_train
    .select(["customer_id", "item_id"])
    .sample(
        n=100_000,
        with_replacement=False,
        seed=42
    )

)


In [ ]:
import polars as pl
import numpy as np

# pos: đã có customer_id, item_id, Y=1
pos = pos_sampled.select(["customer_id", "item_id"]).with_columns(pl.lit(1).alias("Y"))

# Lấy danh sách để sample
train_users = pos["customer_id"].unique().to_list()
all_items = pos["item_id"].unique().to_list()  # hoặc từ items nếu có

# Tạo negative
neg_ratio = 4
n_neg = len(pos) * neg_ratio

neg = pl.DataFrame({
    "customer_id": np.random.choice(train_users, n_neg, replace=True),
    "item_id": np.random.choice(all_items, n_neg, replace=True),
    "Y": 0
})

# === SỬA LỖI TẠI ĐÂY ===
neg = neg.with_columns([
    pl.col("customer_id").cast(pl.Int64),
    pl.col("item_id").cast(pl.Int64),
    pl.col("Y").cast(pl.Int8)
])

pos = pos.with_columns([
    pl.col("customer_id").cast(pl.Int64),
    pl.col("item_id").cast(pl.Int64),
    pl.col("Y").cast(pl.Int8)
])
# ======================

# Ghép và unique
train_pairs = pl.concat([pos, neg]).unique(subset=["customer_id", "item_id"])

n_pos_final = train_pairs.filter(pl.col("Y") == 1).height
n_neg_final = train_pairs.height - n_pos_final

print(f"Positive: {n_pos_final}")
print(f"Negative: {n_neg_final}")
print(f"Total: {train_pairs.height} | Ratio ~1:{n_neg_final//n_pos_final}")

In [ ]:
print(train_pairs.head())

In [ ]:

begin_hist     = datetime(2024, 1, 1)
end_hist       = datetime(2024, 11, 30)
hist = purchase_pl_pre.filter(
        (pl.col("created_date") >= begin_hist) &
        (pl.col("created_date") <= end_hist)
    )

In [ ]:
import polars as pl

def create_feature_table(hist: pl.DataFrame,
                         items: pl.DataFrame,
                         train_pairs: pl.DataFrame) -> pl.DataFrame:
    """
    Tạo bảng feature hoàn chỉnh cho cả positive và negative samples.

    Parameters:
    - hist: DataFrame lịch sử giao dịch thực tế (có cột: customer_id, item_id, quantity, price hoặc không)
    - items: DataFrame thông tin item (có cột: item_id, brand, category_id, price)
    - train_pairs: DataFrame có cột: customer_id, item_id, Y (1=positive, 0=negative)

    Returns:
    - train_final: DataFrame đã được join đầy đủ feature
    """

    # 1. Chuẩn bị items_feat
    items_feat = item_pl_pre.select([
        "item_id", "brand", "category_id", "price"
    ]).with_columns(
        pl.col("price").fill_null(0)
    )

    # 2. Join hist với items để có brand, category, price
    hist_join = hist.join(items_feat, on="item_id", how="left")

    # Tính total_spend cho mỗi giao dịch
    if "quantity" in hist_join.columns:
        hist_join = hist_join.with_columns(
            (pl.col("quantity") * pl.col("price")).alias("total_spend")
        )
    else:
        hist_join = hist_join.with_columns(
            pl.col("price").alias("total_spend")
        )

    # 3. Tính các bảng lịch sử (chỉ từ dữ liệu thực)
    brand_hist = (
        hist_join
        .group_by(["customer_id", "brand"])
        .agg(pl.count().alias("brand_freq"))
    )

    cate_hist = (
        hist_join
        .group_by(["customer_id", "category_id"])
        .agg(pl.count().alias("cate_freq"))
    )

    item_hist = (
        hist_join
        .group_by("item_id")
        .agg(pl.count().alias("item_freq"))
    )

    user_stats = (
        hist_join
        .group_by("customer_id")
        .agg(
            pl.count().alias("user_freq"),
            pl.sum("total_spend").alias("total_spend_user")
        )
        .with_columns(
            (pl.col("total_spend_user") / pl.col("user_freq")).alias("user_avg_spend")
        )
        .select(["customer_id", "user_freq", "user_avg_spend"])
    )

    # 4. Join tất cả feature vào train_pairs (cả pos và neg)
    train_final = (
        train_pairs
        # Join thông tin item
        .join(items_feat, on="item_id", how="left")

        # Join các feature lịch sử
        .join(brand_hist, on=["customer_id", "brand"], how="left")
        .join(cate_hist, on=["customer_id", "category_id"], how="left")
        .join(item_hist, on="item_id", how="left")
        .join(user_stats, on="customer_id", how="left")

        # Tính price_diff
        .with_columns(
            (pl.col("user_avg_spend") - pl.col("price")).alias("price_diff")
        )

        # Fill null → 0 (negative sẽ có nhiều 0, rất tốt!)
        .with_columns([
            pl.col("brand_freq").fill_null(0),
            pl.col("cate_freq").fill_null(0),
            pl.col("item_freq").fill_null(0),
            pl.col("user_freq").fill_null(0),
            pl.col("user_avg_spend").fill_null(0),
            pl.col("price_diff").fill_null(0),
            pl.col("price").fill_null(0),
        ])

        # Chọn cột cuối cùng
        .select([
            "customer_id",
            "item_id",
            "brand_freq",
            "cate_freq",
            "item_freq",
            "price",
            "user_freq",
            "user_avg_spend",
            "price_diff",
            "Y"
        ])
    )

    return train_final

In [ ]:
import polars as pl

def create_feature_table(hist: pl.DataFrame,
                         items: pl.DataFrame,
                         train_pairs: pl.DataFrame) -> pl.DataFrame:
    """
    Tạo bảng feature hoàn chỉnh cho cả positive và negative samples.

    Parameters:
    - hist: DataFrame lịch sử giao dịch thực tế (customer_id, item_id, quantity?, ...)
    - items: DataFrame thông tin item (item_id, brand, category_id, price)
    - train_pairs: DataFrame có cột customer_id (i64), item_id (i64), Y (i8)

    Returns:
    - train_final: DataFrame đầy đủ feature cho cả pos và neg
    """

    # 1. Chuẩn bị items_feat và ÉP KIỂU item_id về Int64 để khớp với train_pairs
    items_feat = (
        item_pl_pre
        .with_columns(
            pl.col("item_id").cast(pl.Int64)  # <-- Quan trọng: ép kiểu item_id thành i64
        )
        .select([
            "item_id", "brand", "category_id", "price"
        ])
        .with_columns(
            pl.col("price").fill_null(0)
        )
    )

    # 2. Join hist với items để lấy brand, category, price
    hist_join = hist.join(items_feat, on="item_id", how="left")

    # Tính total_spend
    if "quantity" in hist_join.columns:
        hist_join = hist_join.with_columns(
            (pl.col("quantity") * pl.col("price")).alias("total_spend")
        )
    else:
        hist_join = hist_join.with_columns(
            pl.col("price").alias("total_spend")
        )

    # 3. Tính các bảng feature lịch sử (dùng pl.len() thay pl.count())
    brand_hist = (
        hist_join
        .group_by(["customer_id", "brand"])
        .agg(pl.len().alias("brand_freq"))
    )

    cate_hist = (
        hist_join
        .group_by(["customer_id", "category_id"])
        .agg(pl.len().alias("cate_freq"))
    )

    item_hist = (
        hist_join
        .group_by("item_id")
        .agg(pl.len().alias("item_freq"))
    )

    user_stats = (
        hist_join
        .group_by("customer_id")
        .agg(
            pl.len().alias("user_freq"),
            pl.sum("total_spend").alias("total_spend_user")
        )
        .with_columns(
            (pl.col("total_spend_user") / pl.col("user_freq")).alias("user_avg_spend")
        )
        .select(["customer_id", "user_freq", "user_avg_spend"])
    )

    # 4. Join feature vào toàn bộ train_pairs (cả pos và neg)
    train_final = (
        train_pairs
        # Join item info (bây giờ item_id đã cùng kiểu i64)
        .join(items_feat, on="item_id", how="left")

        # Join các feature lịch sử
        .join(brand_hist, on=["customer_id", "brand"], how="left")
        .join(cate_hist, on=["customer_id", "category_id"], how="left")
        .join(item_hist, on="item_id", how="left")
        .join(user_stats, on="customer_id", how="left")

        # Tính price_diff
        .with_columns(
            (pl.col("user_avg_spend") - pl.col("price")).alias("price_diff")
        )

        # Fill null → 0
        .with_columns([
            pl.col("brand_freq").fill_null(0),
            pl.col("cate_freq").fill_null(0),
            pl.col("item_freq").fill_null(0),
            pl.col("user_freq").fill_null(0),
            pl.col("user_avg_spend").fill_null(0),
            pl.col("price_diff").fill_null(0),
            pl.col("price").fill_null(0),
        ])

        # Chọn cột cuối
        .select([
            "customer_id",
            "item_id",
            "brand_freq",
            "cate_freq",
            "item_freq",
            "price",
            "user_freq",
            "user_avg_spend",
            "price_diff",
            "Y"
        ])
    )

    return train_final

In [ ]:
import polars as pl

def create_feature_table(hist: pl.DataFrame,
                         items: pl.DataFrame,
                         train_pairs: pl.DataFrame) -> pl.DataFrame:
    """
    Tạo bảng feature, xử lý kiểu item_id để tránh SchemaError.
    """

    # Ép kiểu item_id trong items về Int64
    items_fixed = item_pl_pre.with_columns(
        pl.col("item_id").cast(pl.Int64, strict=False)
    )

    items_feat = items_fixed.select([
        "item_id", "brand", "category_id", "price"
    ]).with_columns(
        pl.col("price").fill_null(0).cast(pl.Float64)
    )

    # Ép kiểu item_id trong hist về Int64 (SỬA CHÍNH Ở ĐÂY)
    hist_fixed = hist.with_columns(
        pl.col("item_id").cast(pl.Int64, strict=False)
    )

    # Giờ join an toàn
    hist_join = hist_fixed.join(items_feat, on="item_id", how="left")

    # Tính total_spend (giữ nguyên)
    if "quantity" in hist_join.columns:
        hist_join = hist_join.with_columns(
            (pl.col("quantity").cast(pl.Float64) * pl.col("price")).alias("total_spend")
        )
    else:
        hist_join = hist_join.with_columns(
            pl.col("price").alias("total_spend")
        )

    # Phần tính feature lịch sử (giữ nguyên, dùng pl.len())
    brand_hist = (
        hist_join
        .group_by(["customer_id", "brand"])
        .agg(pl.len().alias("brand_freq"))
    )

    cate_hist = (
        hist_join
        .group_by(["customer_id", "category_id"])
        .agg(pl.len().alias("cate_freq"))
    )

    item_hist = (
        hist_join
        .group_by("item_id")
        .agg(pl.len().alias("item_freq"))
    )

    user_stats = (
        hist_join
        .group_by("customer_id")
        .agg(
            pl.len().alias("user_freq"),
            pl.sum("total_spend").alias("total_spend_user")
        )
        .with_columns(
            (pl.col("total_spend_user") / pl.col("user_freq")).alias("user_avg_spend")
        )
        .select(["customer_id", "user_freq", "user_avg_spend"])
    )

    # Join vào train_pairs (giữ nguyên)
    train_final = (
        train_pairs
        .join(items_feat, on="item_id", how="left")
        .join(brand_hist, on=["customer_id", "brand"], how="left")
        .join(cate_hist, on=["customer_id", "category_id"], how="left")
        .join(item_hist, on="item_id", how="left")
        .join(user_stats, on="customer_id", how="left")
        .with_columns(
            (pl.col("user_avg_spend") - pl.col("price")).alias("price_diff")
        )
        .with_columns([
            pl.col("brand_freq").fill_null(0),
            pl.col("cate_freq").fill_null(0),
            pl.col("item_freq").fill_null(0),
            pl.col("user_freq").fill_null(0),
            pl.col("user_avg_spend").fill_null(0),
            pl.col("price_diff").fill_null(0),
            pl.col("price").fill_null(0),
        ])
        .select([
            "customer_id",
            "item_id",
            "brand_freq",
            "cate_freq",
            "item_freq",
            "price",
            "user_freq",
            "user_avg_spend",
            "price_diff",
            "Y"
        ])
    )

    return train_final

In [ ]:
train_pairs = train_pairs.with_columns([
    pl.col("item_id").cast(pl.Int64, strict=False),
    pl.col("customer_id").cast(pl.Int64, strict=False),
    pl.col("Y").cast(pl.Int8)
])
train_final = create_feature_table(hist=hist, items=item_pl_pre, train_pairs=train_pairs)

print(train_final.head(10))
print(f"Tổng mẫu: {train_final.height}")
print(f"Positive: {train_final.filter(pl.col('Y') == 1).height}")
print(f"Negative: {train_final.filter(pl.col('Y') == 0).height}")https://colab.research.google.com/drive/1c8O5Kku6Rd1LPSk-Vg672-ExBZ9zQSEC#

/kaggle/input/recomendation-final-groundtrue/final_groundtruth.pkl

In [ ]:
train_final = pd.read_csv("/kaggle/input/train-final/train_final.csv")

In [ ]:
import lightgbm as lgb
from lightgbm import LGBMRanker
import polars as pl
import numpy as np
import gc
from datetime import timedelta
import json

# ==================== [4] TRAINING WITH SEED BLENDING (LGBMRanker) ====================
print("\n[4] TRAINING SEED BLENDING WITH LGBMRanker...")

# Giả sử bạn đã có train_final (có cột customer_id, item_id, Y=1/0)
# và đã sort theo customer_id để group đúng thứ tự
X_train = train_final
feat_cols = [
    "brand_freq", "cate_freq", "item_freq", "price",
    "user_freq", "user_avg_spend", "price_diff"
]

# Group sizes: số lượng candidates mỗi user
groups = X_train.groupby("customer_id").size().to_numpy()

# Seed blending
seeds = [42]
models = []

for seed in seeds:
    print(f"  Training model with seed = {seed}...")
    model = LGBMRanker(
        objective="lambdarank",
        metric="ndcg",
        n_estimators=500,           # tăng số cây để ranking tốt hơn
        learning_rate=0.05,
        num_leaves=128,
        random_state=seed,
        n_jobs=-1,
        verbosity=-1
    )

    model.fit(
        X_train[feat_cols],
        X_train["Y"],
        group=groups
    )
    models.append(model)

print(f"Training completed! {len(models)} models trained.")

# Dọn bộ nhớ
del X_train, groups
gc.collect()

# ==================== [5] PREDICTING... ====================


In [ ]:
import polars as pl

train_final = pl.from_pandas(train_final)


In [ ]:
items = item_stats.head(2500)

In [ ]:
def create_feature_table(hist: pl.DataFrame,
                         items: pl.DataFrame,
                         train_pairs: pl.DataFrame) -> pl.DataFrame:
    """
    Tạo bảng feature ở eager mode (DataFrame) để tránh lỗi Lazy vs DataFrame.
    """
    # 1. items_feat (eager)
    items_feat = items.with_columns(
        pl.col("item_id").cast(pl.Int64, strict=False)
    ).select(["item_id", "brand", "category_id", "price"]).with_columns(
        pl.col("price").fill_null(0)
    )

    # 2. hist_fixed (eager)
    hist_fixed = hist.with_columns(
        pl.col("item_id").cast(pl.Int64, strict=False)
    )

    # Join (eager)
    hist_join = hist_fixed.join(items_feat, on="item_id", how="left")

    # Tính total_spend
    if "quantity" in hist_join.columns:
        hist_join = hist_join.with_columns(
            (pl.col("quantity") * pl.col("price")).alias("total_spend")
        )
    else:
        hist_join = hist_join.with_columns(pl.col("price").alias("total_spend"))

    # 3. Tính feature lịch sử (eager)
    brand_hist = hist_join.group_by(["customer_id", "brand"]).agg(pl.len().alias("brand_freq"))
    cate_hist = hist_join.group_by(["customer_id", "category_id"]).agg(pl.len().alias("cate_freq"))
    item_hist = hist_join.group_by("item_id").agg(pl.len().alias("item_freq"))
    user_stats = hist_join.group_by("customer_id").agg(
        pl.len().alias("user_freq"),
        pl.sum("total_spend").alias("total_spend_user")
    ).with_columns(
        (pl.col("total_spend_user") / pl.col("user_freq")).alias("user_avg_spend")
    ).select(["customer_id", "user_freq", "user_avg_spend"])

    # 4. Join feature vào train_pairs (eager)
    train_final = (
        train_pairs
        .join(items_feat, on="item_id", how="left")
        .join(brand_hist, on=["customer_id", "brand"], how="left")
        .join(cate_hist, on=["customer_id", "category_id"], how="left")
        .join(item_hist, on="item_id", how="left")
        .join(user_stats, on="customer_id", how="left")
        .with_columns(
            (pl.col("user_avg_spend") - pl.col("price")).alias("price_diff")
        )
        .with_columns([
            pl.col("brand_freq").fill_null(0),
            pl.col("cate_freq").fill_null(0),
            pl.col("item_freq").fill_null(0),
            pl.col("user_freq").fill_null(0),
            pl.col("user_avg_spend").fill_null(0),
            pl.col("price_diff").fill_null(0),
            pl.col("price").fill_null(0),
        ])
        .select([
            "customer_id",
            "item_id",
            "brand_freq",
            "cate_freq",
            "item_freq",
            "price",
            "user_freq",
            "user_avg_spend",
            "price_diff",
            "Y"
        ])
    )

    return train_final

In [ ]:
import lightgbm as lgb
from lightgbm import LGBMRanker
import polars as pl
import numpy as np
import gc
from datetime import datetime
import json

print("\n[5] PREDICTING...")

# ==================== Dùng purchase_pl_pre làm positive_pl (toàn bộ lịch sử) ====================
positive_pl = purchase_pl_pre  # ← Đây là toàn bộ lịch sử giao dịch (Polars DataFrame)

# ==================== Kiểm tra cột created_date an toàn (tránh warning) ====================
schema_names = positive_pl.collect_schema().names()  # nhanh, không warning
has_created_date = "created_date" in schema_names

# ==================== 1. hist_dict: item user đã mua ====================
hist_dict = (
    positive_pl
    .group_by("customer_id")
    .agg(pl.col("item_id").unique().alias("bought_items"))
    .collect()  # collect để chuyển sang eager
    .to_pandas()
)
hist_dict = dict(zip(hist_dict["customer_id"], hist_dict["bought_items"]))

# ==================== 2. recent_dict: 10 item gần nhất ====================
if has_created_date:
    recent_pl = (
        positive_pl.sort("created_date", descending=True)
        .group_by("customer_id")
        .agg(pl.col("item_id").head(10).alias("recent_items"))
        .collect()
    )
else:
    recent_pl = (
        positive_pl.group_by("customer_id")
        .agg(pl.col("item_id").head(10).alias("recent_items"))
        .collect()
    )

recent_dict = recent_pl.select(["customer_id", "recent_items"]).to_pandas()
recent_dict = dict(zip(recent_dict["customer_id"], recent_dict["recent_items"]))

# ==================== 3. global_trend: top 50 item phổ biến (sửa đúng cú pháp) ====================
global_trend = (
    positive_pl
    .group_by("item_id")
    .agg(pl.len().alias("cnt"))  # ← sửa pl.count() → pl.len()
    .sort("cnt", descending=True)
    .head(50)
    .select("item_id")
    .collect()  # collect để lấy giá trị
    .get_column("item_id")
    .to_list()  # ← đúng cách lấy list từ cột
)

# ==================== 4. all_users ====================
all_users = positive_pl.select("customer_id").unique().collect().get_column("customer_id").to_list()

# Feature columns
feat_cols = [
    "brand_freq", "cate_freq", "item_freq", "price",
    "user_freq", "user_avg_spend", "price_diff"
]


[5] PREDICTING...


In [ ]:
def create_feature_table(hist: pl.DataFrame,
                         items: pl.DataFrame,
                         train_pairs: pl.DataFrame) -> pl.DataFrame:
    """
    Tạo bảng feature ở eager mode (DataFrame) – an toàn, không lỗi Lazy/DataFrame.
    """
    # Sử dụng tham số 'items' thay vì biến toàn cục 'item_pl_pre'
    items_feat = items.with_columns(
        pl.col("item_id").cast(pl.Int64, strict=False)
    ).select([
        "item_id", "brand", "category_id", "price"
    ]).with_columns(
        pl.col("price").fill_null(0).cast(pl.Float64)
    )

    # hist đã là DataFrame (eager)
    hist_fixed = hist.with_columns(
        pl.col("item_id").cast(pl.Int64, strict=False)
    )

    # Join an toàn (cả hai đều là DataFrame)
    hist_join = hist_fixed.join(items_feat, on="item_id", how="left")

    # Tính total_spend
    if "quantity" in hist_join.columns:
        hist_join = hist_join.with_columns(
            (pl.col("quantity").cast(pl.Float64) * pl.col("price")).alias("total_spend")
        )
    else:
        hist_join = hist_join.with_columns(pl.col("price").alias("total_spend"))

    # Tính các bảng feature
    brand_hist = hist_join.group_by(["customer_id", "brand"]).agg(pl.len().alias("brand_freq"))
    cate_hist = hist_join.group_by(["customer_id", "category_id"]).agg(pl.len().alias("cate_freq"))
    item_hist = hist_join.group_by("item_id").agg(pl.len().alias("item_freq"))

    user_stats = hist_join.group_by("customer_id").agg(
        pl.len().alias("user_freq"),
        pl.sum("total_spend").alias("total_spend_user")
    ).with_columns(
        (pl.col("total_spend_user") / pl.col("user_freq")).alias("user_avg_spend")
    ).select(["customer_id", "user_freq", "user_avg_spend"])

    # Join vào train_pairs
    result = (
        train_pairs
        .join(items_feat, on="item_id", how="left")
        .join(brand_hist, on=["customer_id", "brand"], how="left")
        .join(cate_hist, on=["customer_id", "category_id"], how="left")
        .join(item_hist, on="item_id", how="left")
        .join(user_stats, on="customer_id", how="left")
        .with_columns(
            (pl.col("user_avg_spend") - pl.col("price")).alias("price_diff")
        )
        .with_columns([
            pl.col("brand_freq").fill_null(0),
            pl.col("cate_freq").fill_null(0),
            pl.col("item_freq").fill_null(0),
            pl.col("user_freq").fill_null(0),
            pl.col("user_avg_spend").fill_null(0),
            pl.col("price_diff").fill_null(0),
            pl.col("price").fill_null(0),
        ])
        .select([
            "customer_id",
            "item_id",
            "brand_freq",
            "cate_freq",
            "item_freq",
            "price",
            "user_freq",
            "user_avg_spend",
            "price_diff",
            "Y"
        ])
    )

    return result

In [ ]:
import pickle
import numpy as np

# ==================== Load groundtruth ====================
pkl_path = "/kaggle/input/recomendation-final-groundtrue/final_groundtruth.pkl"
with open(pkl_path, "rb") as f:
    groundtruth = pickle.load(f)

print(groundtruth.head())
print(f"Total users in groundtruth: {len(groundtruth)}")

# Chuyển groundtruth thành dict: user -> list item (giống preds)
gt_dict = groundtruth.set_index("customer_id")["item_id"].to_dict()

# Đảm bảo tất cả item là int và unique
for user in gt_dict:
    if isinstance(gt_dict[user], str):
        import ast
        gt_dict[user] = ast.literal_eval(gt_dict[user])
    gt_dict[user] = list(set(map(int, gt_dict[user])))  # ép int + unique

# preds là dict bạn đã tạo ở phần inference: {user: [item1, item2, ...]}
# hist_dict: {user: list item đã mua trong history} - dùng để filter bought items

# ==================== Hàm đánh giá Precision@K (đã sửa hoàn chỉnh) ====================
def precision_at_k(pred, gt, hist=None, filter_bought_items=True, K=10):
    precisions = []
    precisions_active = []
    precisions_cold = []
    cold_start_users = []
    active_users_count = 0
    cold_start_count = 0

    nusers = len(gt)
    evaluated_users = 0

    for user in gt.keys():
        # Bỏ qua user không có prediction (rất hiếm nếu bạn đã fallback)
        if user not in pred:
            cold_start_users.append(user)
            cold_start_count += 1
            continue

        # Lấy ground truth items
        gt_items = set(gt[user])

        # Xác định user là active hay cold start
        is_active = user in hist and len(hist[user]) > 0
        if is_active:
            active_users_count += 1
        else:
            cold_start_count += 1
            cold_start_users.append(user)

        # Filter bought items nếu cần (chỉ tính item mới là relevant)
        relevant_items = gt_items.copy()
        if filter_bought_items and is_active:
            relevant_items -= set(hist[user])

        # Nếu không còn relevant item nào → precision = 0
        if len(relevant_items) == 0:
            precision = 0.0
        else:
            # Tính hits
            predicted_topk = pred[user][:K]
            hits = len(set(predicted_topk) & relevant_items)
            precision = hits / K

        precisions.append(precision)
        if is_active:
            precisions_active.append(precision)
        else:
            precisions_cold.append(precision)

        evaluated_users += 1

    mean_precision = np.mean(precisions) if precisions else 0.0
    mean_precision_active = np.mean(precisions_active) if precisions_active else 0.0
    mean_precision_cold = np.mean(precisions_cold) if precisions_cold else 0.0

    print(f"\n=== ĐÁNH GIÁ PRECISION@{K} ===")
    print(f"Tổng user trong groundtruth       : {nusers}")
    print(f"User được đánh giá (có prediction): {evaluated_users}")
    print(f"Active users (có lịch sử mua)     : {active_users_count}")
    print(f"Cold start users                  : {cold_start_count}")
    print(f"→ Precision@{K} (Active)          : {mean_precision_active*100:.2f}%")
    print(f"→ Precision@{K} (Cold start)      : {mean_precision_cold*100:.2f}%")
    print(f"→ Precision@{K} (Overall)         : {mean_precision*100:.2f}%")
    return {
        "precision_active": mean_precision_active,
        "precision_cold": mean_precision_cold,
        "precision_overall": mean_precision,
        "n_active": active_users_count,
        "n_cold": cold_start_count,
        "n_evaluated": evaluated_users
    }

# ==================== Chạy đánh giá ====================
results = precision_at_k(
    pred=preds,           # dict từ phần inference
    gt=gt_dict,           # dict từ groundtruth
    hist=hist_dict,       # dict lịch sử mua
    filter_bought_items=True,
    K=10
)

   customer_id                                            item_id
0      2337685                                    [0020010000305]
1      7934799                                    [0020010000438]
2      2052333  [3513000000064, 2403000000004, 3460000000018, ...
3      6548920                                    [6701000000004]
4       368770  [5420000000003, 5420000000002, 1371000000002, ...
Total users in groundtruth: 644970

=== ĐÁNH GIÁ PRECISION@10 ===
Tổng user trong groundtruth       : 644970
User được đánh giá (có prediction): 55529
Active users (có lịch sử mua)     : 55529
Cold start users                  : 589441
→ Precision@10 (Active)          : 2.98%
→ Precision@10 (Cold start)      : 0.00%
→ Precision@10 (Overall)         : 2.98%


In [ ]:
def precision_at_k(pred, gt, hist, filter_bought_items=True, K=10): # prediction, ground-truth, history items, candidate items
    precisions = []
    ideal_precs = []
    ncold_start = 0
    cold_start_users = []
    nusers = len(gt.keys())
    for user in gt.keys():
        if (user not in hist) or (user not in pred):
            ncold_start += 1
            cold_start_users.append(user) # THINKING: để giảm cold start có thể tăng khoảng HISTORY
            continue
        gt_items = gt[user]['list_items']
        relevant_items = set(gt_items)
        if filter_bought_items:
            relevant_items -=set(hist[user])
        # Compute precision@k
        hits = len(set(pred[user][:K]) & relevant_items)
        precisions.append(hits / K)
    return np.mean(precisions), cold_start_users





TypeError: list indices must be integers or slices, not str

In [ ]:
import pickle
import polars as pl
import numpy as np
import gc
from datetime import datetime
import json

# ==================== Load groundtruth (tập test cần đánh giá) ====================
pkl_path = "/kaggle/input/recomendation-final-groundtrue/final_groundtruth.pkl"
with open(pkl_path, "rb") as f:
    groundtruth = pickle.load(f)

print(groundtruth.head())
print(f"Total users in groundtruth: {len(groundtruth)}")

# Chuyển groundtruth thành dict: user -> list item (int, unique)
gt_dict = groundtruth.set_index("customer_id")["item_id"].to_dict()
for user in gt_dict:
    if isinstance(gt_dict[user], str):
        import ast
        gt_dict[user] = ast.literal_eval(gt_dict[user])
    gt_dict[user] = list(set(map(int, gt_dict[user])))

# ==================== Xác định active / cold start users từ purchase_pl_pre ====================
# purchase_pl_pre là toàn bộ lịch sử mua (Polars DataFrame)
active_users_set = set(
    purchase_pl_pre.select("customer_id").unique().collect().get_column("customer_id").to_list()
)

# Danh sách user cần predict: CHỈ những user trong groundtruth
all_users = list(gt_dict.keys())

# Phân loại ngay từ đầu
active_users = [u for u in all_users if u in active_users_set]
cold_users = [u for u in all_users if u not in active_users_set]

print(f"Active users in groundtruth: {len(active_users)}")
print(f"Cold start users in groundtruth: {len(cold_users)}")

# ==================== Chuẩn bị hist_dict, recent_dict, global_trend từ purchase_pl_pre ====================
positive_pl = purchase_pl_pre  # toàn bộ lịch sử

hist_dict = (
    positive_pl
    .group_by("customer_id")
    .agg(pl.col("item_id").unique().alias("bought_items"))
    .select(["customer_id", "bought_items"])
    .collect()
    .to_pandas()
)
hist_dict = dict(zip(hist_dict["customer_id"], hist_dict["bought_items"]))

# recent_dict: 10 item gần nhất (nếu có created_date)
schema_names = positive_pl.collect_schema().names()
has_created_date = "created_date" in schema_names

if has_created_date:
    recent_pl = positive_pl.sort("created_date", descending=True)
else:
    recent_pl = positive_pl

recent_dict = (
    recent_pl
    .group_by("customer_id")
    .agg(pl.col("item_id").head(10).alias("recent_items"))
    .select(["customer_id", "recent_items"])
    .collect()
    .to_pandas()
)
recent_dict = dict(zip(recent_dict["customer_id"], recent_dict["recent_items"]))

# global_trend: top 50 item phổ biến toàn lịch sử
global_trend = (
    positive_pl
    .group_by("item_id")
    .agg(pl.len().alias("cnt"))
    .sort("cnt", descending=True)
    .head(50)
    .collect()
    .get_column("item_id")
    .to_list()
)

   customer_id                                            item_id
0      2337685                                    [0020010000305]
1      7934799                                    [0020010000438]
2      2052333  [3513000000064, 2403000000004, 3460000000018, ...
3      6548920                                    [6701000000004]
4       368770  [5420000000003, 5420000000002, 1371000000002, ...
Total users in groundtruth: 644970
Active users in groundtruth: 485300
Cold start users in groundtruth: 159670


In [ ]:


# ==================== Inference chỉ cho user trong groundtruth ====================
preds = {}
BATCH = 5000  # nhỏ hơn để nhanh

print("Generating candidates and predicting for groundtruth users only...")
for i in range(0, len(all_users), BATCH):
    batch_u = all_users[i:i + BATCH]
    print(f"  Processing batch {i//BATCH + 1} ({len(batch_u)} users)")

    # Candidate Generation
    c_rep_rows = []
    for u in batch_u:
        if u in hist_dict:  # chỉ active mới có repeater
            bought = hist_dict[u][:30]
            c_rep_rows.extend([(u, item) for item in bought])
    c_rep = pl.DataFrame(c_rep_rows, schema=["customer_id", "item_id"], orient="row")

    c_pop = pl.DataFrame({"customer_id": batch_u}).join(
        pl.DataFrame({"item_id": global_trend[:20]}), how="cross"
    )

    full_cands = pl.concat([c_rep, c_pop], how="diagonal").unique(subset=["customer_id", "item_id"])

    if full_cands.height == 0:
        continue

    # Build features
    candidate_with_y = full_cands.with_columns(
        pl.col("item_id").cast(pl.Int64),
        pl.lit(0).alias("Y")
    )

    feat_pl = create_feature_table(
        hist=positive_pl.lazy(),
        items=item_pl_pre.lazy(),
        train_pairs=candidate_with_y.lazy()
    )

    feat_df = feat_pl.select(["customer_id", "item_id"] + feat_cols).collect().to_pandas()

    if feat_df.empty:
        continue

    # Prediction (ensemble nếu có nhiều model)
    feat_df["score"] = model.predict(feat_df[feat_cols])

    top_candidates = feat_df.sort_values(["customer_id", "score"], ascending=[True, False]) \
                            .groupby("customer_id").head(20)

    # Heuristic blending
    for u, group in top_candidates.groupby("customer_id"):
        model_items = group["item_id"].tolist()
        recents = recent_dict.get(u, [])[:5]

        final = []
        seen = set()

        for x in recents:
            if x not in seen:
                final.append(x)
                seen.add(x)

        for x in model_items:
            if len(final) >= 10:
                break
            if x not in seen:
                final.append(x)
                seen.add(x)

        for x in global_trend:
            if len(final) >= 10:
                break
            if x not in seen:
                final.append(x)
                seen.add(x)

        preds[u] = final[:10]

    # Fallback cho cold start users (không có candidate từ model)
    for u in batch_u:
        if u not in preds:
            preds[u] = global_trend[:10]

    del full_cands, candidate_with_y, feat_pl, feat_df, top_candidates, c_rep, c_pop
    gc.collect()




Generating candidates and predicting for groundtruth users only...
  Processing batch 1 (5000 users)


/tmp/ipykernel_47/338921265.py:25: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "quantity" in hist_join.columns:


  Processing batch 2 (5000 users)


/tmp/ipykernel_47/338921265.py:25: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "quantity" in hist_join.columns:


  Processing batch 3 (5000 users)


...

/tmp/ipykernel_47/338921265.py:25: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "quantity" in hist_join.columns:


  Processing batch 128 (5000 users)


/tmp/ipykernel_47/338921265.py:25: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "quantity" in hist_join.columns:


  Processing batch 129 (4970 users)


/tmp/ipykernel_47/338921265.py:25: PerformanceWarning: Determining the column names of a LazyFrame requires resolving its schema, which is a potentially expensive operation. Use `LazyFrame.collect_schema().names()` to get the column names without this warning.
  if "quantity" in hist_join.columns:


In [ ]:
# ==================== Đánh giá Precision@10 ====================
def precision_at_k(pred, gt, active_set, K=10):
    precisions = []
    precisions_active = []
    precisions_cold = []

    for user in gt.keys():
        if user not in pred:
            precision = 0.0
        else:
            gt_items = set(gt[user])
            predicted = pred[user][:K]

            # Filter bought items cho active user
            if user in active_set:
                bought = set(hist_dict.get(user, []))
                relevant = gt_items - bought
            else:
                relevant = gt_items  # cold start không filter

            if len(relevant) == 0:
                precision = 0.0
            else:
                hits = len(set(predicted) & relevant)
                precision = hits / K

        precisions.append(precision)
        if user in active_set:
            precisions_active.append(precision)
        else:
            precisions_cold.append(precision)

    mean_all = np.mean(precisions)
    mean_active = np.mean(precisions_active) if precisions_active else 0.0
    mean_cold = np.mean(precisions_cold) if precisions_cold else 0.0

    print(f"\n=== PRECISION@{K} TRÊN GROUNDTRUTH ===")
    print(f"Tổng user đánh giá       : {len(gt)}")
    print(f"Active users             : {len(active_users)} → Precision@{K}: {mean_active*100:.3f}%")
    print(f"Cold start users         : {len(cold_users)} → Precision@{K}: {mean_cold*100:.3f}%")
    print(f"Overall                  : {mean_all*100:.3f}%")

    return mean_active, mean_cold, mean_all


In [ ]:
# Chạy đánh giá
precision_at_k(
    pred=preds,
    gt=gt_dict,
    active_set=set(active_users),
    K=10
)


=== PRECISION@10 TRÊN GROUNDTRUTH ===
Tổng user đánh giá       : 644970
Active users             : 485300 → Precision@10: 5.538%
Cold start users         : 159670 → Precision@10: 1.439%
Overall                  : 4.523%


(0.05538100144240676, 0.014387173545437464, 0.045232491433710095)

In [ ]:
# Lưu submission nếu cần
clean_preds = {str(k): [str(x) for x in v] for k, v in preds.items()}
with open("submission_groundtruth_only.json", "w") as f:
    json.dump(clean_preds, f)